# Shopee Best Coders 2019 (Undergraduate) Round 2 - Fraud Detection

In this notebook, I will attempt to solve this challenge https://www.kaggle.com/c/opn-rd2-acv/overview using Graph data structure.

## 1. Reading users data into dataframes

In [1]:
import pandas as pd

# Reading users' Bank Account, Credit Card, Device and Orders data from csv
bank_accounts = pd.read_csv(r".\src\static\bank_accounts.csv")
credit_cards = pd.read_csv(r".\src\static\credit_cards.csv")
devices = pd.read_csv(r".\src\static\devices.csv")
orders = pd.read_csv(r".\src\static\orders.csv")

# Displaying dataframes
display(bank_accounts)
display(credit_cards)
display(devices)
display(orders)

C:\Users\justi\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\justi\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\justi\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)
C:\Users\justi\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,userid,bank_account
0,21829134,923302000003892
1,95910542,11002023212822
2,96941876,189303223
3,23452396,2280003199803
4,12647942,32002028484803
...,...,...
350836,23568930,084902023320802
350837,17207934,023102020991822
350838,23862272,0090343142
350839,23983072,3330993800


,userid,credit_card
0,2579938,832299xxxxxx4902|02-2019
1,2154902,322429xxxxxx3848|05-2020
2,5578604,322429xxxxxx1293|11-2016
3,2051728,388329xxxxxx4303|09-2016
4,6216212,322421xxxxxx4238|05-2018
...,...,...
38703,25879226,839199xxxxxx3838|12-2022
38704,9331492,398138xxxxxx2208|08-2019
38705,2890486,839330xxxxxx9180|08-2022
38706,107429252,899933xxxxxx2930|06-2021


,userid,device
0,10499978,0Zu/mWZ4cFsvobYglgZEc5VSxwwajRwrm74pBuwnrTu=
1,54526742,zBMRKyz98iy5Z7sh+JIOdF612J1CB8ggvFBI6MP361q=
2,33152428,KXC0oY3MMrTG+bnlFFvBPXZC5PW8iw6kgWztENWdIJg=
3,55020986,anRXS7+zVgCGKz5C9X3dzewIhSdojGEi5TO07pb2y+K=
4,100471502,hbIARr+USwvjjzc8QUVtD5a/apR1DJX6P3fYvgDL4gW=
...,...,...
1430306,7061456,ZSTXD6wfHN/1qPVqMrIbwHYX5ELWlUPO3NFXCHK50Mc=
1430307,26416970,o+9gbX201ITS3MuMZYjuZUzV7sOoR57/0ttyX3eSWTA=
1430308,79633116,NZhNiXYhArpPquzrqh/kVj2IZ3QPRKm6FH90vsTlhiy=
1430309,19883370,dZ7YqVTprtTvLT4qS0YEWsn9EJahBJVQtTLKC6WaHZA=


,orderid,buyer_userid,seller_userid
0,1953278092,47388162,20822974
1,1953295120,26855196,16416890
2,1953306402,121296714,28477978
3,1953314712,131221930,72837912
4,1953381964,183398314,28423332
...,...,...,...
620942,1956636054,14222102,70940136
620943,1956642760,2978154,60766142
620944,1956651496,221974772,15541938
620945,1956660812,152533590,80986700


## 2. Setting up a graph of 'user_ids' called 'users_graph'

In [2]:
# !python -m pip install networkx
import networkx as nx

# Create a graph with 'user_id' as nodes and edges to connect
# 'user_ids' that share the same 'bank_account', 'credit_card' or 'device'
users_graph = nx.Graph()

def add_user_id_link(G, user_id1, user_id2, attr1, attr2):
    """
    Function to add link in graph G for user_id1 and user_id2
    if they share the same 'attr' ('device', 'credit_card' or 'bank_account')
    
    Args:
    - G: Empty graph or existing graph with user_ids as nodes and edges
         connecting users sharing the same 'bank_account', 'credit_card' 
         or 'device'
    - user_id1, user_id2: string denoting the first and second user_id
        to connect.
    - attr1, attr2: string attribute by which we check the connection
        between user_id1 and user_id2

    Returns:
    - None
    """
    if attr1 == attr2:
        G.add_edge(user_id1, user_id2)

for df in [bank_accounts, credit_cards, devices]:
    # Ensure that 'bank_account', 'credit_card' and 'device' are str
    df.iloc[:,1] = df.iloc[:,1].astype('str')

    # Sorting by 'bank_account', 'credit_card' and 'device'
    # and compare if consecutive rows have the same 'user_id'
    df.sort_values(df.columns[1], inplace=True)

    # Add shifted rows as new columns to easily compare consecutive rows
    df_next_row = df.shift(1)
    df_next_row.columns = df.columns + '2'
    
    df_combined = pd.concat([df, df_next_row], axis='columns')

    # If consecutive rows (columns 1 & 2 vs 3 & 4) share the same
    # 'bank_account', 'credit_card' or 'device', add link on users_graph
    df_combined.apply(lambda row: add_user_id_link(users_graph, row['userid'], row['userid2'], row.values[1], row.values[3]), axis=1)

## 3. Detecting fraudulent activities using users_graph

In [3]:
def is_fraud(G, buyer_id, seller_id):    
    """
    Function to detect frauds by checking if there exists a path 
    from buyer_id node to seller_id node in users_graph.

    Args:
    - G: Graph with user_ids as nodes and edges connecting users
        sharing the same 'bank_account', 'credit_card' or 'device'
    - buyer_id: the user_id of the buyer in the transaction
    - seller_id: the seller_id of the seller in the transaction

    Returns:
    - 1 if a transaction is a fraud and 0 otherwise
    """
    # Check if both buyer_id and seller_id are in users_graph
    if G.has_node(buyer_id) and G.has_node(seller_id):
        # Check if there is a path from buyer_id to seller_id
        if buyer_id in nx.algorithms.descendants(G, seller_id):
            return 1
    return 0

orders['is_fraud'] = orders.apply(lambda row: is_fraud(users_graph, row.buyer_userid, row.seller_userid), axis=1)
display(orders)

,orderid,buyer_userid,seller_userid,is_fraud
0,1953278092,47388162,20822974,0
1,1953295120,26855196,16416890,0
2,1953306402,121296714,28477978,0
3,1953314712,131221930,72837912,0
4,1953381964,183398314,28423332,0
...,...,...,...,...
620942,1956636054,14222102,70940136,0
620943,1956642760,2978154,60766142,0
620944,1956651496,221974772,15541938,0
620945,1956660812,152533590,80986700,0


## 4. Saving orderids and fraudulent flags as csv

In [4]:
df_frauds = orders[['orderid', 'is_fraud']]
df_frauds.to_csv(r'.\output\ans.csv', index=False)